In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
import tensorflow as tf
import random

In [ ]:
import io
import cv2
from scipy.ndimage import rotate
from nibabel import load
from IPython.display import Image, display
from skimage.exposure import rescale_intensity
from skimage.segmentation import mark_boundaries
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, Callback, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    #Contraction path
    c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='bce', metrics=[dice_coef, tpr, fpr])    
    return model

In [ ]:
predictions_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/cylinder_niipredictions/'
raw_image_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/raw_images/'

raw_images = sorted(os.listdir(raw_image_directory))
predictions = sorted(os.listdir(predictions_directory))

unet_prediction_dataset = []
raw_image_dataset = []
sliced_unet_prediction_dataset = []
sliced_raw_image_dataset = []
image_sizes = []

for image_name in predictions:    
    if (image_name.split('.')[1] == 'nii'):
        base_name = image_name.split('.')[0]
        image = nib.load(predictions_directory+image_name).get_fdata()
        raw = nib.load(raw_image_directory+image_name).get_fdata()
        unet_prediction_dataset.append(np.array(image))
        raw_image_dataset.append(np.array(raw))
        image_sizes.append(image.shape[2])

for i in range(len(unet_prediction_dataset)):
    for j in range(unet_prediction_dataset[i].shape[-1]):
        sliced_raw_image_dataset.append(raw_image_dataset[i][:,:,j])
        prediction = unet_prediction_dataset[i][:,:,:,:,j]
        prediction = prediction.squeeze(0)
        sliced_unet_prediction_dataset.append(prediction)

In [ ]:
for i in range(len(unet_prediction_dataset)):
    prediction = unet_prediction_dataset[i].squeeze(0)
    middle = unet_prediction_dataset[i].shape[-1] // 2
    raw_middle = raw_image_dataset[i].shape[-1] // 2
    plt.figure(figsize=(16, 8))
    plt.subplot(131)
    plt.title('Raw Slice')
    plt.imshow(raw_image_dataset[i][:,:,raw_middle], cmap='gray')
    plt.axis('off')
    plt.subplot(132)
    plt.title('Prediction')
    plt.imshow(prediction[:,:,0,middle], cmap='gray')
    plt.axis('off')
    plt.subplot(133)
    plt.title('Overlay')
    plt.imshow(raw_image_dataset[i][:,:,raw_middle], cmap='gray')
    plt.imshow(prediction[:,:,0,middle], cmap='grey', alpha=0.3)
    plt.axis('off')
    plt.show()
    plt.close()

In [ ]:
# START FROM HERE!!

raw_image_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/raw_images/'
ground_truth_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/segmentations/'
unet_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/cylinder_niipredictions/'
# unetpp_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/unet++_predictions/'

raw_images = sorted(os.listdir(raw_image_directory))
segmentations = sorted(os.listdir(ground_truth_directory))
unet_predictions = sorted(os.listdir(unet_directory))
# unetpp_predictions = sorted(os.listdir(unetpp_directory))

image_dataset = []
mask_dataset = []
unet_prediction_dataset = []
# unetpp_prediction_dataset = []
image_names = []

for image_name in raw_images:    
    if (image_name.split('.')[1] == 'nii'):
        base_name = image_name.split('.')[0]
        image = nib.load(raw_image_directory+image_name).get_fdata()
        segmentation = nib.load(ground_truth_directory+image_name).get_fdata()
        new_mask = np.copy(segmentation)
        new_mask[segmentation == 1] = 0
        unet = nib.load(unet_directory+image_name).get_fdata()
        # unetpp = nib.load(unetpp_directory+image_name).get_fdata()
        
        image_dataset.append(np.array(image))
        mask_dataset.append(np.array(new_mask))
        unet_prediction_dataset.append(np.array(unet))
        # unetpp_prediction_dataset.append(np.array(unetpp))
        image_names.append(base_name)

In [ ]:
for i in range(len(unet_prediction_dataset)):
    for j in range(image_dataset[i].shape[2]):
        plt.figure(figsize=(16, 8))
        plt.subplot(121)
        plt.title(f'{image_names[i]}_slice{j}')
        plt.imshow(image_dataset[i][:,:,j], cmap='gray')
        plt.axis('off')
        plt.subplot(122)
        plt.title('Ground Truth Slice')
        plt.imshow(mask_dataset[i][:,:,j], cmap='gray')
        plt.axis('off')
        plt.show()

In [ ]:
num_classes = 7
stats = {}

# allComparisons = {"GroundTruth": mask_dataset, "Unet": unet_prediction_dataset, "Unet++":unetpp_prediction_dataset}
allComparisons = {"GroundTruth": mask_dataset, "Unet": unet_prediction_dataset}
slices = {"Mid-Slice+1":1, "Mid-Slice":0, "Mid-Slice-1":-1}

for i in range(len(image_dataset)):
    image = image_dataset[i]
    image_name = image_names[i]
    mid_slice = image.shape[2]//2
    for slice_label, offset in slices.items():
        row_key = f"{image_name}_{slice_label}"
        stats[row_key] = {}
        for model_name, model_masks in allComparisons.items():
            if model_name == 'GroundTruth':
                for j in range(num_classes-1):
                    mask_class =model_masks[i]==(j+2)
                    slice_data = (mask_class*image)[:,:,mid_slice+offset] 
                    slice_data = slice_data[slice_data!=0]
                    mean = np.mean(slice_data) if slice_data.size>0 else 0
                    std = np.std(slice_data) if slice_data.size>0 else 0
                    stats[row_key][model_name, f"Cylinder {j+1}", "Mean"] = mean
                    stats[row_key][model_name, f"Cylinder {j+1}", "Std"] = std
            else:
                for j in range(num_classes-2):                   
                    if len(model_masks[i].shape) != 3:
                        mask_class = model_masks[i].squeeze(0)
                        mask_class = (mask_class[:,:,j+1,:] > 0.5).astype(int)
                        slice_data = (mask_class*image)[:,:,mid_slice+offset] 
                        slice_data = slice_data[slice_data!=0]
                        mean = np.mean(slice_data) if slice_data.size>0 else 0
                        std = np.std(slice_data) if slice_data.size>0 else 0
                        stats[row_key][model_name, f"Cylinder {j+1}", "Mean"] = mean
                        stats[row_key][model_name, f"Cylinder {j+1}", "Std"] = std
                    else:
                        mask_class = (model_masks[i]==(j+1) > 0.5).astype(int)
                        slice_data = (mask_class*image)[:,:,mid_slice+offset] 
                        slice_data = slice_data[slice_data!=0]
                        mean = np.mean(slice_data) if slice_data.size>0 else 0
                        std = np.std(slice_data) if slice_data.size>0 else 0
                        stats[row_key][model_name, f"Cylinder {j+1}", "Mean"] = mean
                        stats[row_key][model_name, f"Cylinder {j+1}", "Std"] = std

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(stats, orient='index')
df.columns = pd.MultiIndex.from_tuples(df.columns, names=['Model', 'Cylinder', 'Stat'])
df.index.name = "Image_Slice"
df = df.drop(columns=[('GroundTruth', 'Cylinder 6', 'Mean'), ('GroundTruth', 'Cylinder 6', 'Std')])
df

In [ ]:
gt_df = df.xs('GroundTruth', axis=1, level='Model')
unet_df = df.xs('Unet', axis=1, level='Model')

gtAllZero = (gt_df==0).all(axis=1)
gtAllNonZero = (gt_df!=0).all(axis=1)
unetAllZero = (unet_df==0).all(axis=1)
unetAllNonZero = (unet_df!=0).all(axis=1)
df = df[(gtAllNonZero & unetAllNonZero) | (gtAllZero & unetAllZero)]
df

In [ ]:
# Save DataFrame to an Excel file
df.to_excel('stats4_dataframe.xlsx', index=True)  # Include index (file names)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

all_data = pd.read_excel('stats4_dataframe.xlsx')
all_data = all_data[3:]
all_data.columns = (['image_name',
'ground_truth_1_mean', 'ground_truth_1_std', 
'ground_truth_2_mean', 'ground_truth_2_std', 
'ground_truth_3_mean', 'ground_truth_3_std', 
'ground_truth_4_mean', 'ground_truth_4_std', 
'ground_truth_5_mean', 'ground_truth_5_std',
'unet_1_mean', 'unet_1_std',
'unet_2_mean', 'unet_2_std',
'unet_3_mean', 'unet_3_std',
'unet_4_mean', 'unet_4_std',
'unet_5_mean', 'unet_5_std',])
all_data['unet_difference_1'] = all_data['unet_1_mean'] - all_data['ground_truth_1_mean']
all_data['unet_difference_2'] = all_data['unet_2_mean'] - all_data['ground_truth_2_mean']
all_data['unet_difference_3'] = all_data['unet_3_mean'] - all_data['ground_truth_3_mean']
all_data['unet_difference_4'] = all_data['unet_4_mean'] - all_data['ground_truth_4_mean']
all_data['unet_difference_5'] = all_data['unet_5_mean'] - all_data['ground_truth_5_mean']
all_data['unet_std_difference_1'] = all_data['unet_1_std'] - all_data['ground_truth_1_std']
all_data['unet_std_difference_2'] = all_data['unet_2_std'] - all_data['ground_truth_2_std']
all_data['unet_std_difference_3'] = all_data['unet_3_std'] - all_data['ground_truth_3_std']
all_data['unet_std_difference_4'] = all_data['unet_4_std'] - all_data['ground_truth_4_std']
all_data['unet_std_difference_5'] = all_data['unet_5_std'] - all_data['ground_truth_5_std']
all_data['unet_average_1'] = (all_data['unet_1_mean'] + all_data['ground_truth_1_mean']) /2
all_data['unet_average_2'] = (all_data['unet_2_mean'] + all_data['ground_truth_2_mean']) /2
all_data['unet_average_3'] = (all_data['unet_3_mean'] + all_data['ground_truth_3_mean']) /2
all_data['unet_average_4'] = (all_data['unet_4_mean'] + all_data['ground_truth_4_mean']) /2
all_data['unet_average_5'] = (all_data['unet_5_mean'] + all_data['ground_truth_5_mean']) /2

differences = all_data[['image_name', 'unet_difference_1', 'unet_difference_2', 'unet_difference_3', 'unet_difference_4', 'unet_difference_5', 'unet_std_difference_1', 'unet_std_difference_2', 'unet_std_difference_3', 'unet_std_difference_4', 'unet_std_difference_5', 'unet_average_1', 'unet_average_2', 'unet_average_3', 'unet_average_4', 'unet_average_5']]
differences.set_index('image_name', inplace=True)
differences

In [ ]:
cylinder_1 = differences[['unet_difference_1']]
cylinder_2 = differences[['unet_difference_2']]
cylinder_3 = differences[['unet_difference_3']]
cylinder_4 = differences[['unet_difference_4']]
cylinder_5 = differences[['unet_difference_5']]

upper_outliers_1 = (cylinder_1 < 13).all(axis=1)
lower_outliers_1 = (cylinder_1 > -10).all(axis=1)

upper_outliers_2 = (cylinder_2 < 19).all(axis=1)
lower_outliers_2 = (cylinder_2 > -2).all(axis=1)

upper_outliers_3 = (cylinder_3 < 18).all(axis=1)
lower_outliers_3 = (cylinder_3 > -6).all(axis=1)

upper_outliers_4 = (cylinder_4 < 17).all(axis=1)
lower_outliers_4 = (cylinder_4 > 0).all(axis=1)

upper_outliers_5 = (cylinder_5 < 20).all(axis=1)
lower_outliers_5 = (cylinder_5 > -10).all(axis=1)


differences = differences[(upper_outliers_1 & lower_outliers_1)]
differences = differences[(upper_outliers_2 & lower_outliers_2)]
differences = differences[(upper_outliers_3 & lower_outliers_3)]
differences = differences[(upper_outliers_4 & lower_outliers_4)]
differences = differences[(upper_outliers_5 & lower_outliers_5)]

differences

In [ ]:
mean_difference1 = differences['unet_difference_1'].mean()
std_difference1 = differences['unet_difference_1'].std()

mean_difference2 = differences['unet_difference_2'].mean()
std_difference2 = differences['unet_difference_2'].std()

mean_difference3 = differences['unet_difference_3'].mean()
std_difference3 = differences['unet_difference_3'].std()

mean_difference4 = differences['unet_difference_4'].mean()
std_difference4 = differences['unet_difference_4'].std()

mean_difference5 = differences['unet_difference_5'].mean()
std_difference5 = differences['unet_difference_5'].std()

loa_upper1 = mean_difference1 + 1.96 * std_difference1
loa_lower1 = mean_difference1 - 1.96 * std_difference1

loa_upper2 = mean_difference2 + 1.96 * std_difference2
loa_lower2 = mean_difference2 - 1.96 * std_difference2

loa_upper3 = mean_difference3 + 1.96 * std_difference3
loa_lower3 = mean_difference3 - 1.96 * std_difference3

loa_upper4 = mean_difference4 + 1.96 * std_difference4
loa_lower4 = mean_difference4 - 1.96 * std_difference4

loa_upper5 = mean_difference5 + 1.96 * std_difference5
loa_lower5 = mean_difference5 - 1.96 * std_difference5

mean_difference = (mean_difference1 + mean_difference2 + mean_difference3 + mean_difference4 + mean_difference5)/5
mean_loa_upper = (loa_upper1 + loa_upper2 + loa_upper3 + loa_upper4 + loa_upper5)/5
mean_loa_lower = (loa_lower1 + loa_lower2 + loa_lower3 + loa_lower4 + loa_lower5)/5

plt.figure(figsize=(10, 8))
plt.scatter(differences['unet_average_1'], differences['unet_difference_1'], color='red', label='Cylinder 1')
plt.scatter(differences['unet_average_2'], differences['unet_difference_2'], color='blue', label='Cylinder 2')
plt.scatter(differences['unet_average_3'], differences['unet_difference_3'], color='green', label='Cylinder 3')
plt.scatter(differences['unet_average_4'], differences['unet_difference_4'], color='orange', label='Cylinder 4')
plt.scatter(differences['unet_average_5'], differences['unet_difference_5'], color='purple', label='Cylinder 5')
plt.axhline(y=mean_difference, color='black', linestyle='--', label='Mean difference')
plt.axhline(y=mean_loa_upper, color='red', linestyle='--', label='Mean Upper limit of agreement (LOA)')
plt.axhline(y=mean_loa_lower, color='red', linestyle='--', label='Mean Lower limit of agreement (LOA)')

# Adding labels and title
plt.xlabel('Average of True and Predicted Values')
plt.ylabel('Difference (Predicted - True)')
plt.title('Bland-Altman Pooled Plot')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(differences['unet_average_1'], differences['unet_difference_1'], color='red', label='Cylinder 1')
plt.axhline(y=mean_difference1, color='black', linestyle='--', label='Mean difference')
plt.axhline(y=loa_upper1, color='red', linestyle='--', label='Mean Upper limit of agreement (LOA)')
plt.axhline(y=loa_lower1, color='red', linestyle='--', label='Mean Lower limit of agreement (LOA)')

# Adding labels and title
plt.xlabel('Average of True and Predicted Values')
plt.ylabel('Difference (Predicted - True)')
plt.title('Bland-Altman Plot Cylinder 1')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(differences['unet_average_2'], differences['unet_difference_2'], color='blue', label='Cylinder 2')
plt.axhline(y=mean_difference2, color='black', linestyle='--', label='Mean difference')
plt.axhline(y=loa_upper2, color='red', linestyle='--', label='Mean Upper limit of agreement (LOA)')
plt.axhline(y=loa_lower2, color='red', linestyle='--', label='Mean Lower limit of agreement (LOA)')

# Adding labels and title
plt.xlabel('Average of True and Predicted Values')
plt.ylabel('Difference (Predicted - True)')
plt.title('Bland-Altman Plot Cylinder 2')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(differences['unet_average_3'], differences['unet_difference_3'], color='green', label='Cylinder 3')
plt.axhline(y=mean_difference3, color='black', linestyle='--', label='Mean difference')
plt.axhline(y=loa_upper3, color='red', linestyle='--', label='Mean Upper limit of agreement (LOA)')
plt.axhline(y=loa_lower3, color='red', linestyle='--', label='Mean Lower limit of agreement (LOA)')

# Adding labels and title
plt.xlabel('Average of True and Predicted Values')
plt.ylabel('Difference (Predicted - True)')
plt.title('Bland-Altman Plot Cylinder 3')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(differences['unet_average_4'], differences['unet_difference_4'], color='orange', label='Cylinder 4')
plt.axhline(y=mean_difference4, color='black', linestyle='--', label='Mean difference')
plt.axhline(y=loa_upper4, color='red', linestyle='--', label='Mean Upper limit of agreement (LOA)')
plt.axhline(y=loa_lower4, color='red', linestyle='--', label='Mean Lower limit of agreement (LOA)')

# Adding labels and title
plt.xlabel('Average of True and Predicted Values')
plt.ylabel('Difference (Predicted - True)')
plt.title('Bland-Altman Plot Cylinder 4')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(differences['unet_average_5'], differences['unet_difference_5'], color='purple', label='Cylinder 5')
plt.axhline(y=mean_difference5, color='black', linestyle='--', label='Mean difference')
plt.axhline(y=loa_upper5, color='red', linestyle='--', label='Mean Upper limit of agreement (LOA)')
plt.axhline(y=loa_lower5, color='red', linestyle='--', label='Mean Lower limit of agreement (LOA)')

# Adding labels and title
plt.xlabel('Average of True and Predicted Values')
plt.ylabel('Difference (Predicted - True)')
plt.title('Bland-Altman Plot Cylinder 5')
plt.legend()
plt.show()

In [ ]:
print("Cylinder 1: ")
print("Mean Difference: ", mean_difference1)
print("Limits of Agreement: ", loa_upper1, loa_lower1)
print(loa_upper1 - mean_difference1)

print("Cylinder 2: ")
print("Mean Difference: ", mean_difference2)
print("Limits of Agreement: ", loa_upper2, loa_lower2)
print(loa_upper2 - mean_difference2)

print("Cylinder 3: ")
print("Mean Difference: ", mean_difference3)
print("Limits of Agreement: ", loa_upper3, loa_lower3)
print(loa_upper3 - mean_difference3)

print("Cylinder 4: ")
print("Mean Difference: ", mean_difference4)
print("Limits of Agreement: ", loa_upper4, loa_lower4)
print(loa_upper4 - mean_difference4)

print("Cylinder 5: ")
print("Mean Difference: ", mean_difference5)
print("Limits of Agreement: ", loa_upper5, loa_lower5)
print(loa_upper5 - mean_difference5)

print("Average: ")
print("Mean Difference: ", mean_difference)
print("Limits of Agreement: ", mean_loa_upper, mean_loa_lower)
print(mean_loa_upper - mean_difference)

In [ ]:
mean_std_difference = (std_difference1 + std_difference2 + std_difference3 + std_difference4 + std_difference5)/5
mean_std_difference